# Avian data 2022-2023
This notebook performs and document the transformations performed on the data received from Colibri by The Water Institute as part of this project. 

The main goals of this transformations are:
  - Create a unified dataset for 2010-2023 data (This notebook will only process the data from 2022-2023 but using the schema from the previous years)
  - Rename both high resolution photos and screenshots to have them in a unified folder structure
  - Create thumbnails for the high resolution photos to be visualized on the web dashboard
  - Generate datasets for being used in the visualizations

In [ ]:
#%pip install tqdm pandas boto3
import getpass

### Parameters

In [ ]:
thumbnail_size = (518, 345)
create_thumbnails = True
rename_files=False
replace = False
_base_folder = "avian_monitoring"
role_to_assume = getpass.getpass("Input the role")


### Common imports

In [ ]:
from PIL import Image
import re
import pandas as pd

# import geopandas as gpd
import pandas_access
from datetime import datetime
import pyodbc
import numpy as np
from multiprocessing import Pool
from functools import partial
import geopandas as gp
import boto3
import re
from tqdm.notebook import tqdm
from dateutil.parser import parse
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO
from functools import partial

tqdm.pandas()

boto3.setup_default_session(profile_name="GLO")
bucket_name = "twi-aviandata"
starting_folder = "HighResolutionImages"
new_folder_hr = "avian_monitoring_"
sts_client = boto3.client('sts')
assumed_role_object=sts_client.assume_role(
    RoleArn= role_to_assume,
    RoleSessionName="AssumeRoleSession1",
    DurationSeconds=43000
)
credentials=assumed_role_object['Credentials']

aws_s3 = boto3.resource(
    "s3",
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken'],
)
avian_data = aws_s3.Bucket(bucket_name)

### Utilities

In [ ]:
def clean_date(text):
    """Return a date as yyyy-MMM-dd (e.g. 2022-May-15)"""
    datetimestr = parse(text)
    text = datetime.strftime(datetimestr, "%Y-%b-%d")
    return text


def update_mime_type(t, mime="image/png"):
    """Update the metadata of s3 objects to the given Mime type"""
    s3_object = avian_data.Object(t)
    s3_object.copy_from(
        CopySource={"Bucket": avian_data.name, "Key": t},
        Metadata=s3_object.metadata,
        MetadataDirective="REPLACE",
        ContentType=mime,
    )


def copy_wnew_mime_type(t, k, mime="image/png"):
    """Copy a s3 object, modifying its mime type"""
    s3_object = avian_data.Object(t)
    avian_data.copy(
        {"Bucket": avian_data.name, "Key": t},
        k,
        Metadata=s3_object.metadata,
        MetadataDirective="REPLACE",
        ContentType=mime,
    )

def rename(key, new_name, replace=False):
    """Copy an object (key) to a new location (new_name) on the avian_data bucket"""
    if not replace and [o for o in avian_data.objects.filter(Prefix=new_name)]:
        return True

    try:
        avian_data.copy({"Bucket": avian_data.name, "Key": key}, new_name)
    except Exception as e:
        print(e)
        check_lowercase = list(
            avian_data.objects.filter(Prefix=key.replace(".JPG", ".jpg"))
        )
        if check_lowercase:
            avian_data.copy(
                {"Bucket": avian_data.name, "Key": check_lowercase[0].key}, new_name
            )
            return True
        check_uppercase = list(
            avian_data.objects.filter(Prefix=key.replace(".jpg", ".JPG"))
        )
        if check_uppercase:
            avian_data.copy(
                {"Bucket": avian_data.name, "Key": check_uppercase[0].key}, new_name
            )
            return True
        print(f"{key} not found")
        return False


def generate_thumbnail(high_res_key, thumb_key, regenerate=False):
    """Generate a low resolution thumbnail at thumb_key of the high_res_key object.
    it uses the global thumbnail_size to define the size.
    """
    current = avian_data.objects.filter(Prefix=thumb_key)
    if not regenerate and [o for o in current]:
        return True
    objs = [o for o in avian_data.objects.filter(Prefix=high_res_key)]
    if not objs:
        print(f"there is no {high_res_key}")
        return False
    image = Image.open(BytesIO(objs[0].get()["Body"].read()))
    buffer = BytesIO()
    image.thumbnail(thumbnail_size)
    image.save(buffer, format="png")
    buffer.seek(0)
    avian_data.put_object(Key=thumb_key, Body=buffer, ContentType="image/png")
    return True

def createListings(tree, path):
    """Recursive method to list all files and folders in a tree,
    with the format required by jquery browse files"""
    dirs = []
    files = []
    listings = {}
    for e in tree:
        if tree[e]:
            dirs.append(e)
            listings.update(createListings(tree[e], path + "/" + e))
        elif "." in e:
            files.append(e)

    listings[path] = {"dirs": dirs, "files": files}
    return listings

def exists_key(key, replace=False):
    try:
        return len(list(avian_data.objects.filter(Prefix=key))) > 0
    except Exception as e:
        print(f"key:{key}, exception {e}")
        return False



# Unified 2010-2021 Data

Data has been unified by Colibri into a single access database, however it contains tables discriminated by years, with some differences in the schema. This process will combine all the datasets into a single one with a common deffinition of the total birds and total nests. 

In [ ]:
# Notes: In order to use all the fields, I did a rename of the columns containing '?' or '/'
acc_db = "Colibri2010-2021CWBColonies_2Jan2023.accdb"

schema = pandas_access.read_schema(acc_db)


In [ ]:
ct_name = "tblRWCWB_ColonyInventory_10Nov22"
# colonies_table = schema.pop("tblRWCWB_ColonyInventory_13Sept2022")
colonies_table = schema.pop(ct_name)


There are three tables with slighly different schema

In [ ]:
schema


Taking the 2015 to 2021 table as a reference, lets compare the schemas.

Fields in the reference but not in the `tblSpeciesData2011_2013` table

In [ ]:
schema["tblSpeciesData2015_2018_2021"].keys() - schema["tblSpeciesData2011_2013"].keys()


Fields in the `tblSpeciesData2011_2013` table but not in the reference

In [ ]:
schema["tblSpeciesData2011_2013"].keys() - schema["tblSpeciesData2015_2018_2021"].keys()

Fields in the referece but not in the 2010 table

In [ ]:
schema["tblSpeciesData2015_2018_2021"].keys() - schema["tblSpeciesData2010"].keys()


Fields in the referece but not in the 2010 table

In [ ]:
schema["tblSpeciesData2010"].keys() - schema["tblSpeciesData2015_2018_2021"].keys()


That also means that we have different formulas to calculate the total. The bestForBPE field is used as filter in 2013 to 2021, but it is not used in the 2010.

### Formulas

|                | 2010                                                                                                                                                               | 2011-2013                                                                                                                                                          | 2015-2021                                                                                                 |
|----------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|
| Nests          | sum(<br>[WBN]<br>+[ChickNestw/outAdult]<br>+[AbandNest]<br>+[EmptyNest]<br>+[PBN]<br>+[Site]<br>+[Brood])                                                          | Sum(<br>[WBN]<br>+[ChickNestw/outAdult]<br>+[AbandNest]<br>+[EmptyNest]<br>+[PBN]<br>+[Site]<br>+[Brood])                                                          | Sum(<br>[WBN]<br>+[ChickNest]<br>+[ChickNestw/outAdult]<br>+[AbandNest]<br>+[PBN]<br>+[Site]<br>+[Brood]) |
| Birds          | sum([WBN]<br>+[PBN]<br>+[Site]<br>+[OtherAdultsInColony]<br>+[OtherImmInColony]<br>+[RoostingBirds]<br>+[RoostingAdults]<br>+[RoostingImmatures]<br>+[UnknownAge]) | Sum([WBN]<br>+[PBN]<br>+[Site]<br>+[OtherAdultsInColony]<br>+[OtherImmInColony]<br>+[RoostingBirds]<br>+[RoostingAdults]<br>+[RoostingImmatures]<br>+[UnknownAge]) | Sum(<br>[WBN]<br>+[ChickNest]<br>+[PBN]<br>+[Territory]<br>+[Site]<br>+[OtherBirds])                      |
| SumOfEmptyNest | sum(EmptyNest)                                                                                                                                                     | EmptyNest                                                                                                                                                          | EmptyNest                                                                                                 |


But 2015 to 2021 has additional categories: 

```SQL

Sum(WBN) AS SumOfWBN

Sum(ChickNest) AS SumOfChickNest

Sum([ChickNestw/outAdult]) AS [SumOfChickNestw/outAdult] 

Sum(Brood) AS SumOfBrood

Sum(AbandNest) AS SumOfAbandNest

Sum(PBN) AS SumOfPBN

Sum(Territory) AS SumOfTerritory
```

## Generating a common dataset:
- merge 2010 species with the colonies. 
- select only the common columns 

In [ ]:
cols_2015_2021 = set(schema["tblSpeciesData2015_2018_2021"].keys())
cols_2011_2013 = set(schema["tblSpeciesData2011_2013"].keys())
cols_2010 = set(schema["tblSpeciesData2010"].keys())
common_fields = cols_2011_2013.intersection(cols_2015_2021)


In [ ]:
cols_2010 - common_fields


In [ ]:
common_fields - cols_2010


# UPDATE Mime types 

# 2022-2023 data

In [ ]:
acc_db2023 = "./LACWB_2022-2023.accdb"
schema2023 = pandas_access.read_schema(acc_db2023)

In [ ]:
fields2023 = set( schema2023["tblSpeciesData"].keys())

In [ ]:
fields2023

In [ ]:
common_fields - fields2023

In [ ]:
fields2023 - common_fields

In [ ]:
common_fields

In [ ]:
pd_species_2023 = pandas_access.read_table(acc_db2023, "tblSpeciesData")
colonies_table2023 =  pandas_access.read_table(acc_db2023, "RWCWB_ColonyInventory_13Septemb")

In [ ]:
colonies_table2023

In [ ]:
pd_species_2023 = pd.merge(pd_species_2023, colonies_table2023, on="ColonyName", how="left")


In [ ]:
pd_species_2023["HighResImage_new"] = (
    f"{_base_folder}/high_resolution_photos/"
    + pd_species_2023["Year"].astype(str)
    + "/"
    + pd_species_2023["GeoRegion"]
    + "/"
    + pd_species_2023["ColonyName"]
    + "/"
    + pd.to_datetime(
        pd_species_2023["Date"].astype("str"), format="%m/%d/%y %H:%M:%S"
    ).dt.strftime("%d%B%y")
    + "Camera"
    + pd_species_2023["CameraNumber"]
    + "-"
    + "Card"
    + (pd_species_2023["CardNumber"].where(~pd_species_2023["CardNumber"].isnull(), other="1"))
    + "-"
    + pd_species_2023["PhotoNumber"]
    + ".jpg"
)
pd_species_2023["screenshot_new"] = (
    f"{_base_folder}/screenshots/"
    + pd_species_2023["Year"].astype(str)
    + "/"
    + pd_species_2023["GeoRegion"]
    + "/"
    + pd_species_2023["ColonyName"]
    + "/"
    + pd.to_datetime(
        pd_species_2023["Date"].astype("str"), format="%m/%d/%y %H:%M:%S"
    ).dt.strftime("%d%B%y")
    + "Camera"
    + pd_species_2023["CameraNumber"]
    + "-"
    + "Card"
    + (pd_species_2023["CardNumber"].where(~pd_species_2023["CardNumber"].isnull(), other="1"))
    + "-"
    + pd_species_2023["PhotoNumber"]
    + ".jpg"
)
pd_species_2023["thumbnail_new"] = (
    f"{_base_folder}/thumbnails/"
    + pd_species_2023["Year"].astype(str)
    + "/"
    + pd_species_2023["GeoRegion"]
    + "/"
    + pd_species_2023["ColonyName"]
    + "/"
    + pd.to_datetime(
        pd_species_2023["Date"].astype("str"), format="%m/%d/%y %H:%M:%S"
    ).dt.strftime("%d%B%y")
    + "Camera"
    + pd_species_2023["CameraNumber"]
    + "-"
    + "Card"
    + (pd_species_2023["CardNumber"].where(~pd_species_2023["CardNumber"].isnull(), other="1"))
    + "-"
    + pd_species_2023["PhotoNumber"]
    + ".png"
)


# There are some rows without photo information
They are from the locations using the new method

In [ ]:

v  = pd_species_2023[pd_species_2023["HighResImage_new"].isna()][["Year","ColonyName"]].value_counts()

In [ ]:
print(v.sort_index().to_string())

In [ ]:
 pd_species_2023["total_nests"] = (pd_species_2023["WBN"]
    + pd_species_2023["ChickNestwithoutAdult"]
    + pd_species_2023["AbandNest"]
    + pd_species_2023["ChickNest"]
    + pd_species_2023["PBN"]
    + pd_species_2023["Site"]
    + pd_species_2023["Brood"])
pd_species_2023["total_birds"] = (pd_species_2023["WBN"]
    + pd_species_2023["ChickNest"]
    + pd_species_2023["PBN"]
    + pd_species_2023["Territory"]
    + pd_species_2023["Site"]
    + pd_species_2023["OtherBirds"])

In [ ]:
pd_species_2023["date2"] = pd_species_2023["Date"].astype("str").apply(clean_date)
pd_species_2023["month"] = pd_species_2023["date2"].apply(lambda x: x.split("-")[1])
order_by_field = "total_nests" 

In [ ]:
pd_species_2023["SpeciesCode"] = pd_species_2023["SpeciesCode"].str.upper()


In [ ]:
pd_species_2023[pd_species_2023["BestForBPE"]=='Y']

In [ ]:
pd_species_2023["uid"] = (
    np.where(
        pd_species_2023["HighResImage_new"].isna(),
        pd_species_2023["date2"].astype(str)
        +"+"
        +pd_species_2023["ColonyName"],
        pd_species_2023["HighResImage_new"]
    ) 
    + "#"
    + np.where(pd_species_2023["SpeciesCode"].isna(),
               "N/A",
               pd_species_2023["SpeciesCode"])
)

In [ ]:
pd_species_2023.loc[pd_species_2023.SpeciesCode.str.startswith("REEG "),"SpeciesCode"]="REEG"

In [ ]:
pd_species_2023

In [ ]:
pd_species_2023.to_excel("avianmonitoring_2023.xlsx", index=False)


In [ ]:
agg_pd_species = (
    pd_species_2023.loc[
        pd_species_2023.BestForBPE == "Y",
        [
            "Year",
            "Date",
            "State",
            "GeoRegion",
            "ColonyName",
            "SpeciesCode",
            "Longitude",
            "Latitude",
            "total_nests",
            "total_birds",
        ],
    ]
    .groupby(
        [
            "Year",
            "Date",
            "State",
            "GeoRegion",
            "ColonyName",
            "Longitude",
            "Latitude",
            "SpeciesCode"
        ]
    )
    .sum()
)

In [ ]:
agg_pd_species

In [ ]:
agg_pd_species.reset_index().rename(columns={"total_nests": "Nests", "total_birds": "Birds"}).to_excel("SummaryFileGenerated2023.xlsx", index=False)

# Reorganize Files
2022 photos are organized by date camea card and photo number:
eg. `15May2022_Camera1Card1/15May2022_Cam1Card1_1.jpg`
2023 photos have a month folder also, and a small change on naming
`2023/June 2023/21June2023Cam2Card1/21June2023_Cam2Card1_1.jpg`
Following the existing schema, for photos used on the dotting dattabase, we want to organize them in the `f"{_base_folder}/high_resolution_photos/"` folder under a GeoRegion/ColonyName subfolder

In [ ]:
_hr_original_folder = "HighResolutionImages/"
# the candidate will have this format 2022/15May2022_Camera1Card1/15May2022_Cam1Card1_1.jpg
#if year is 2023, then there is a folder for each month 'June 2023' or 'May 2023'

pd_species_2023["candidate_hr"] = (_hr_original_folder
+ pd_species_2023["Year"].astype("str") + "/"
+ (np.where(pd_species_2023["Year"]==2023, pd.to_datetime(
        pd_species_2023["Date"].astype("str"), format="%m/%d/%y %H:%M:%S"
    ).dt.strftime("%B %Y") +"/", ""))
+pd.to_datetime(
        pd_species_2023["Date"].astype("str"), format="%m/%d/%y %H:%M:%S"
    ).dt.strftime("%d%B%Y") 
+ np.where( pd.to_datetime(pd_species_2023["Date"].astype("str"), format="%m/%d/%y %H:%M:%S").dt.strftime("%B %Y")=="June 2023", "Cam","_Camera")
+pd_species_2023["CameraNumber"]
+"Card"
+pd_species_2023["CardNumber"]
+"/"
+pd.to_datetime(
        pd_species_2023["Date"].astype("str"), format="%m/%d/%y %H:%M:%S"
    ).dt.strftime("%d%B%Y")
+"_Cam"
+pd_species_2023["CameraNumber"]
+"Card"
+pd_species_2023["CardNumber"]
+"_"
+pd_species_2023["PhotoNumber"].str.strip()
+".jpg")

m_grouped = pd_species_2023[["HighResImage_new","candidate_hr","thumbnail_new"]].drop_duplicates()
#remove the ones that do not have a HighResImage_new
m_grouped = m_grouped[~m_grouped["HighResImage_new"].isna()]
with ThreadPoolExecutor(max_workers=16) as e:
        futures = list(
            tqdm(
                e.map(
                    lambda x, y: rename(x, y),
                    m_grouped["candidate_hr"].tolist(),
                    m_grouped["HighResImage_new"].tolist(),
                ),
                total=m_grouped.shape[0],
            )
        )

In [ ]:
if create_thumbnails:
    with ThreadPoolExecutor(max_workers=16) as e:
        futures = list(
            tqdm(
                e.map(
                    partial(generate_thumbnail, regenerate=False),
                    m_grouped["HighResImage_new"].tolist(),
                    m_grouped["thumbnail_new"].tolist(),
                ),
                total=m_grouped.shape[0],
            )
        )


for 2023 screenshots, the original files are available at: 
`DottedImages/2023/Breton Screen Captures/` and have the format:`24June23BRETArea1.JPG`
Given that we have the Area and the date on the dataframe we can generate the candidate names

In [ ]:
pd_species_2023

In [ ]:
pd_species_2023["candidate_screenshot"]=np.where((pd_species_2023["Year"]==2023) & (pd_species_2023["ColonyName"]=="Breton Island"),
"DottedImages/2023/Breton Screen Captures/"
+pd.to_datetime(
        pd_species_2023["Date"].astype("str"), format="%m/%d/%y %H:%M:%S"
    ).dt.strftime("%d%B%y")+"BRETArea"+pd_species_2023["DottingAreaNumber"]+".JPG",
                                                 None
)
s_grouped = pd_species_2023.loc[~pd_species_2023.candidate_screenshot.isnull(),["screenshot_new","candidate_screenshot"]].drop_duplicates()


In [ ]:
with ThreadPoolExecutor(max_workers=16) as e:
        futures = list(
            tqdm(
                e.map(
                    lambda x, y: rename(x, y),
                    s_grouped["candidate_screenshot"].tolist(),
                    s_grouped["screenshot_new"].tolist(),
                ),
                total=s_grouped.shape[0],
            )
        )

## Update list of files on AWS (for file browsing)

In [ ]:
files_in_bucket = list(avian_data.objects.all())
tree = {}
filenames = [f.key for f in files_in_bucket if not f.key.endswith("/")]
for file in filenames:
    parent = tree
    filepath = file.split("/")
    for p in filepath:
        level = parent.get(p, {})
        parent[p] = level
        parent = parent[p]

In [ ]:
remove_from_root = ["403.html", 
                    "index.html", 
                    "list_files.html",
                    "list_files_ns.html",
                    "explorer.css",
                    "explorer.js",
                    "browser.html",
                    "test.txt",
                    "image_unavailable.png",
                    ]
for r in remove_from_root:
    if r in tree:
        tree.pop(r)


In [ ]:
tree.keys()

In [ ]:
listings = createListings(tree, "")
import json

with open("file_listing.json", "w") as jsonfile:
    json.dump(listings, jsonfile)

In [ ]:
avian_data.put_object(Key="file_listing.json", Body=json.dumps(listings), ContentType="application/json")

# Check existing files

In [ ]:
# This is equivalent, but faster than: pd_species["thumbnail_new"] = pd_species["thumbnail_new"].progress_apply(exists_key)
with ThreadPoolExecutor(max_workers=128) as e:
    futures = {}
    _grouped = pd_species_2023[
        filter(lambda x: "_new" in x, pd_species_2023.columns)
    ].drop_duplicates()
    for f in _grouped.columns:
        futures[f] = list(
            tqdm(
                e.map(lambda y: exists_key(y), _grouped[f].tolist()),
                total=_grouped.shape[0],
            )
        )

In [ ]:
for x in futures:
    with pd.option_context("display.max_colwidth", None):
        display(_grouped.loc[~np.array(futures[x]), x])


In [ ]:
x = "HighResImage_new"
pd_species_2023[pd_species_2023[x].isin(_grouped.loc[~np.array(futures[x]), x])]


In [ ]:
for x in futures:
    print(x, np.sum(~np.array(futures[x])))
    with pd.option_context("display.max_colwidth", None):
        _no_exists = pd_species_2023[
            pd_species_2023[x].isin(_grouped.loc[~np.array(futures[x]), x])
        ]
        display(_no_exists)
        display(_no_exists[["Year", x]].drop_duplicates().groupby(["Year"]).count())
        pd_species_2023.loc[
            pd_species_2023[x].isin(_grouped.loc[~np.array(futures[x]), x]), x
        ] = None
#   with pd.option_context('display.max_colwidth', None):
#       display(pd_species.loc[~np.array(futures[x]), x].drop_duplicates())
#       pd_species.loc[~np.array(futures[x]), x].drop_duplicates().to_csv(f"not_found_{x}.csv", index=False)

#   display(pd_species.loc[~np.array(futures[x]),[x,"Year"]].drop_duplicates().groupby("Year").count())
#   pd_species.loc[~np.array(futures[x]), x] = None


In [ ]:
pd_species_2023.to_csv("avianmonitoring_2022-2023_Nulls.csv.gz", index=False)


In [ ]:
pd_species_2023.to_excel("avianmonitoring_2022-2023_Nulls.xlsx", index=False)


In [ ]:
if replace and create_thumbnails:
    m_grouped["thumbnail_new"].progress_apply(lambda t: update_mime_type(t))


In [ ]:
if rename_files:
    with ThreadPoolExecutor(max_workers=16) as e:
        futures = list(
            tqdm(
                e.map(
                    lambda t: update_mime_type(t, mime="image/jpeg"),
                    m_grouped["HighResImage_new"].tolist(),
                ),
                total=m_grouped.shape[0],
            )
        )
